In [211]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [244]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
import time
import datetime
from pyspark.sql.functions import *
from pyspark.sql.functions import unix_timestamp
import datetime
from datetime import datetime
#from pyspark.sql.functions import col, udf
#from pyspark.sql.types import DateType

from pyspark.sql import SparkSession


spark = SparkSession.builder.master("local").\
                    appName("Word Count").\
                    config("spark.driver.bindAddress","localhost").\
                    config("spark.ui.port","4040").\
                    getOrCreate()

Создайте схему будущего фрейма данных. Схема должна включать следующие атрибуты:

·   id -  уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. Данное поле не является первичным ключом.

·   timestamp – дата и время события в формате unix timestamp.

·   type – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), перед на другую страницу(move)).

·   page_id – id текущей страницы. Тип - последовательность чисел фиксированной длины.

·   tag – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику конкретной новости со страницы. Возможный список тематик: политика, спорт, медицина и т.д.

·   sign – наличие у пользователя личного кабинета. Значения – True/False.

In [213]:
schema = T.StructType([
                T.StructField("id", T.IntegerType(), True),
                T.StructField("timestamp", T.LongType(), True),
                T.StructField("type", T.StringType(), True),
                T.StructField("page_id", T.IntegerType(), True),
                T.StructField("tag", T.StringType(), True),
                T.StructField("sign", T.BooleanType(), True)
])

Создайте датафрейм с описанной выше схемой данных.

Наполните датафрейм данными. Пример:

(12345, 1667627426, "click", 101, "Sport”, False)

In [266]:
data = [(134441, 1667627426, 'visit', 101, 'политика', True),
        (134441, 1667627500, 'scroll', 101, 'политика', True),
        (134441, 1667627000, 'move', 200, 'политика', True),
        (134442, 1667627426, 'visit', 200, 'политика', True),
        (134442, 1667627500, 'scroll', 101, 'политика', True),
        (134443, 1667627555, 'move', 101, 'политика', False),
        (134443, 1667627426, 'visit', 101, 'политика', False),
        (134443, 1667627500, 'scroll', 101, 'политика', False),
        (134443, 1667627555, 'click', 200, 'политика', False),
        (134444, 1667627426, 'visit', 101, 'политика', False),
        (134441, 1667627500, 'click', 200, 'политика', True),
        (134441, 1667627555, 'move', 101, 'политика', True),
        (134442, 1667627426, 'click', 101, 'политика', True),
        (134442, 1667627500, 'click', 102, 'политика', True),
        (134443, 1667655555, 'move', 101, 'политика', False),
        (134443, 1667655426, 'visit', 103, 'спорт', False),
        (134443, 1667665500, 'scroll', 103, 'спорт', False),
        (134443, 1667665555, 'move', 105, 'политика', False),
        (134443, 1667666555, 'click', 105, 'политика', False),
        (134444, 1667627426, 'visit', 101, 'политика', False),
        (134445, 1667627500, 'scroll', 101, 'политика', True),
        (134446, 1667630000, 'move', 101, 'политика', True),
        (134447, 1667627426, 'visit', 101, 'политика', False),
        (134448, 1667627426, 'visit', 101, 'политика', True),
        (134448, 1667627500, 'scroll', 101, 'политика', True),
        (134448, 1667627555, 'click', 150, 'спорт', True),
        (134448, 1667627500, 'scroll', 101, 'политика', True),
        (134448, 1667627555, 'move', 101, 'политика', True),
        (134449, 1667627555, 'click', 150, 'спорт', True),
        (134450, 1667627555, 'click', 150, 'спорт', True),
        (134450, 1667627555, 'move', 101, 'политика', True),
        (134450, 1667627555, 'click', 101, 'политика', True),
        (134450, 1667627555, 'move', 101, 'политика', True),
        (134448, 1667627500, 'click', 300, 'политика', True),
        (134448, 1667627555, 'click', 300, 'политика', True),
        (134449, 1667627555, 'click', 300, 'политика', True),
        (134450, 1667627555, 'click', 155, 'спорт', True),
        (134450, 1667627555, 'click', 155, 'спорт', True),
        (134450, 1667627555, 'click', 157, 'спорт', True),
        (134450, 1667627555, 'click', 158, 'спорт', True)
]
df = spark.createDataFrame(data = data, schema = schema)

In [215]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- type: string (nullable = true)
 |-- page_id: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- sign: boolean (nullable = true)



In [216]:
df.show()

+------+----------+------+-------+--------+-----+
|    id| timestamp|  type|page_id|     tag| sign|
+------+----------+------+-------+--------+-----+
|134441|1667627426| visit|    101|политика| true|
|134441|1667627500|scroll|    101|политика| true|
|134441|1667627000|  move|    200|политика| true|
|134442|1667627426| visit|    200|политика| true|
|134442|1667627500|scroll|    101|политика| true|
|134443|1667627555|  move|    101|политика|false|
|134443|1667627426| visit|    101|политика|false|
|134443|1667627500|scroll|    101|политика|false|
|134443|1667627555| click|    200|политика|false|
|134444|1667627426| visit|    101|политика|false|
|134441|1667627500| click|    200|политика| true|
|134441|1667627555|  move|    101|политика| true|
|134442|1667627426| click|    101|политика| true|
|134442|1667627500| click|    102|политика| true|
|134443|1667655555|  move|    101|политика|false|
|134443|1667655426| visit|    103|политика|false|
|134443|1667665500|scroll|    103|политика|false|


Вывести топ-5 самых активных посетителей сайта

In [267]:
df.groupBy(col("id")).count().sort(desc("count")).limit(5).show()

+------+-----+
|    id|count|
+------+-----+
|134443|    9|
|134450|    8|
|134448|    7|
|134441|    5|
|134442|    4|
+------+-----+



Посчитать процент посетителей, у которых есть ЛК

In [268]:
df1 = df.select("id","sign").distinct()
df1.show()
df1.where(df1.sign == True).count() / df1.count()

+------+-----+
|    id| sign|
+------+-----+
|134447|false|
|134441| true|
|134449| true|
|134450| true|
|134446| true|
|134445| true|
|134444|false|
|134442| true|
|134443|false|
|134448| true|
+------+-----+



0.7

 Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице

In [269]:
df.filter(df.type == "click").groupBy(col("page_id")).count().sort(desc("count")).show(5)

+-------+-----+
|page_id|count|
+-------+-----+
|    300|    3|
|    150|    3|
|    200|    2|
|    101|    2|
|    155|    2|
+-------+-----+
only showing top 5 rows



Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

In [270]:
df3 = df.withColumn("time_range", when(from_unixtime(col("timestamp"),"hh") <= 4,"0-4")
              .when(from_unixtime(col("timestamp"),"hh") <= 8,"5-8")
              .when(from_unixtime(col("timestamp"),"hh") <= 12,"9-12"))
df3.show(200)

+------+----------+------+-------+--------+-----+----------+
|    id| timestamp|  type|page_id|     tag| sign|time_range|
+------+----------+------+-------+--------+-----+----------+
|134441|1667627426| visit|    101|политика| true|       5-8|
|134441|1667627500|scroll|    101|политика| true|       5-8|
|134441|1667627000|  move|    200|политика| true|       5-8|
|134442|1667627426| visit|    200|политика| true|       5-8|
|134442|1667627500|scroll|    101|политика| true|       5-8|
|134443|1667627555|  move|    101|политика|false|       5-8|
|134443|1667627426| visit|    101|политика|false|       5-8|
|134443|1667627500|scroll|    101|политика|false|       5-8|
|134443|1667627555| click|    200|политика|false|       5-8|
|134444|1667627426| visit|    101|политика|false|       5-8|
|134441|1667627500| click|    200|политика| true|       5-8|
|134441|1667627555|  move|    101|политика| true|       5-8|
|134442|1667627426| click|    101|политика| true|       5-8|
|134442|1667627500| clic

Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.

In [271]:
df3.groupBy(col("time_range")).count().sort(desc("count")).show(1)

+----------+-----+
|time_range|count|
+----------+-----+
|       5-8|   35|
+----------+-----+
only showing top 1 row



Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов

1.       Id – уникальный идентификатор личного кабинета
2.       User_id – уникальный идентификатор посетителя
3.       ФИО посетителя
4.       Дату рождения посетителя
5.       Дата создания ЛК

In [310]:
df_users = spark.createDataFrame([(256888, 134441, "Ricnhinova Ojuna Cidipovna", datetime.date(1985,3,13), datetime.date(2022,5,30)),
                                  (256889, 134442, "Petrov Ivan Vasilevich", datetime.date(1960,10,1), datetime.date(2022,5,21)),
                                  (256890, 134443, "Zaicev Vasil Vasilevich", datetime.date(1991,12,31), datetime.date(2022,5,21)),
                                  (256891, 134444, "Ivanov D D", datetime.date(2000,5,31), datetime.date(2022,6,2)),
                                  (256892, 134445, "Petrov T R", datetime.date(2001,5,15), datetime.date(2022,6,3)),
                                  (256893, 134446, "Zuleiman Z W", datetime.date(1999,8,3), datetime.date(2022,6,3)),
                                  (256894, 134447, "Kim A F", datetime.date(2001,6,10), datetime.date(2022,6,3)),
                                  (256895, 134448, "Cydenov R T", datetime.date(2002,11,7), datetime.date(2022,7,1)),
                                  (256896, 134449, "Cydypova Z R", datetime.date(2003,8,6), datetime.date(2022,7,1)),
                                  (256897, 134450, "Mother S R", datetime.date(2003,9,13), datetime.date(2022,7,1))
]
, ["id", "user_id", "FIO", "birthday_date", "create_date"])

Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.

In [311]:
df.filter(df.tag == "спорт").join(df_users, df.id == df_users.user_id, 'inner').select("FIO").distinct().show()

+--------------------+
|                 FIO|
+--------------------+
|Zaicev Vasil Vasi...|
|        Cydypova Z R|
|          Mother S R|
|         Cydenov R T|
+--------------------+



Выведите 10% ЛК, у которых максимальная разница между датой создания ЛК и датой последнего посещения.

In [418]:
df4 = df.groupBy(col("id")).max("timestamp")
#df4.show()
df5 = df.join(df_users, df.id == df_users.user_id, 'inner').select(df.id.alias("id"), to_date(from_unixtime(df.timestamp,"yyyy-dd-MM")).alias("last_date"), df_users.create_date.alias("create_date")).distinct()
#df6 = df.join(df_users, df.id == df_users.user_id, 'inner').withColumn("diff_time", to_date(from_unixtime(df.timestamp,"yyyy-dd-MM")))
df5.withColumn("diff_time", datediff(df5.create_date, df5.last_date)).sort(desc("diff_time")).limit(int((df5.count()*0.1))).show()
#df6.show(100)

+------+----------+-----------+---------+
|    id| last_date|create_date|diff_time|
+------+----------+-----------+---------+
|134449|2022-05-11| 2022-07-01|       51|
+------+----------+-----------+---------+



Вывести топ-5 страниц, которые чаще всего посещают мужчины и топ-5 страниц, которые посещают чаще женщины.

In [417]:
df_user_sexes = spark.createDataFrame([(134441, "female"),
                                  (134442, "male"),
                                  (134443, "male"),
                                  (134444, "male"),
                                  (134445, "male"),
                                  (134446, "male"),
                                  (134447, "female"),
                                  (134448, "male"),
                                  (134449, "female"),
                                  (134450, "female"),
]
, ["user_id", "sex"])
print("топ-5 страниц, которые чаще всего посещают мужчины")
df.filter(df.type == "visit").join(df_users, df.id == df_users.user_id, 'inner').join(df_user_sexes, df_users.user_id == df_user_sexes.user_id, 'inner').filter(df_user_sexes.sex == "male").groupBy("page_id").count().sort(desc("count")).show(5)
#select("sex", "page_id").show()
print("топ-5 страниц, которые чаще всего посещают женщины")
df.filter(df.type == "visit").join(df_users, df.id == df_users.user_id, 'inner').join(df_user_sexes, df_users.user_id == df_user_sexes.user_id, 'inner').filter(df_user_sexes.sex == "female").groupBy("page_id").count().sort(desc("count")).show(5)


топ-5 страниц, которые чаще всего посещают мужчины


+-------+-----+
|page_id|count|
+-------+-----+
|    101|    4|
|    103|    1|
|    200|    1|
+-------+-----+

топ-5 страниц, которые чаще всего посещают женщины
+-------+-----+
|page_id|count|
+-------+-----+
|    101|    2|
+-------+-----+

